In [1]:
# 导入必要的库
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import torch
import torch.nn as nn
import numpy as np
import random
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import torch.nn.functional as F
import nbimporter
from network import *
import os
from utils import *
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import matplotlib

In [2]:
seed=42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
num_class=4
num_epoch=2500
test_inverval = 50
num_view = 3
dim_hvcdn= 64
batch_size = 125

In [ ]:
# 读取制表符分隔的CSV文件
Methylation_train = pd.read_csv('../../DATA/THCA_parting/1_tr.csv')
# 显示 DataFrame
Methylation_train

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.108356,0.111206,0.212663,0.441164,0.263589,0.373276,0.215142,0.231430,0.415421,0.524971,...,0.563012,0.116748,0.637430,0.042472,0.473440,0.252532,0.519429,0.432116,0.350196,0.163122
1,0.095593,0.092424,0.067180,0.450420,0.283063,0.323043,0.227279,0.196455,0.401015,0.523737,...,0.419127,0.103232,0.566139,0.000000,0.461772,0.219299,0.321675,0.352588,0.311547,0.142307
2,0.155456,0.121434,0.104714,0.471631,0.324398,0.355628,0.241474,0.209778,0.406169,0.520879,...,0.459746,0.149027,0.670796,0.075482,0.537622,0.272693,0.466133,0.420018,0.332122,0.206836
3,0.165101,0.152276,0.113309,0.466523,0.298085,0.335604,0.218271,0.169460,0.420195,0.510951,...,0.519247,0.120110,0.674506,0.000000,0.489361,0.290446,0.497281,0.351045,0.333715,0.170886
4,0.125337,0.134063,0.079028,0.468523,0.201158,0.349485,0.275529,0.211588,0.435993,0.462107,...,0.502319,0.139415,0.587636,0.000000,0.446979,0.214085,0.465480,0.330512,0.300085,0.175081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,0.100223,0.029615,0.138830,0.377822,0.209048,0.439149,0.206836,0.306878,0.454488,0.502880,...,0.500366,0.126615,0.601225,0.158056,0.571851,0.304004,0.468395,0.514099,0.322872,0.305926
206,0.139415,0.092424,0.129764,0.456725,0.302057,0.373519,0.183568,0.199217,0.393360,0.540179,...,0.512199,0.131005,0.624044,0.022105,0.465087,0.234854,0.439694,0.417872,0.288097,0.175540
207,0.109788,0.140578,0.123398,0.483855,0.241474,0.337932,0.275997,0.238201,0.392509,0.498019,...,0.514852,0.109073,0.599984,0.000000,0.471568,0.308950,0.480523,0.342632,0.265855,0.170412
208,0.109788,0.055338,0.109788,0.475411,0.292759,0.379695,0.280820,0.255240,0.380854,0.525627,...,0.491298,0.101735,0.628378,0.000000,0.499588,0.293383,0.396511,0.390899,0.320813,0.241768


In [ ]:
# 读取制表符分隔的CSV文件
Methylation_test = pd.read_csv('../../DATA/THCA_parting/1_te.csv')
# 显示 DataFrame
Methylation_test

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.029615,0.025907,0.110499,0.393784,0.227603,0.294212,0.184434,0.119444,0.445864,0.481583,...,0.456308,0.022105,0.430470,0.000000,0.390250,0.204968,0.463638,0.321331,0.243810,0.164609
1,0.165101,0.083345,0.058388,0.476021,0.300283,0.385066,0.285053,0.202311,0.393148,0.497334,...,0.572936,0.161620,0.724728,0.069999,0.519657,0.298085,0.441087,0.439383,0.350196,0.244100
2,0.128512,0.139415,0.176454,0.484605,0.274825,0.362444,0.271494,0.234237,0.429391,0.531031,...,0.498962,0.138830,0.687974,0.000000,0.502421,0.270044,0.402720,0.428051,0.326075,0.161620
3,0.138830,0.140578,0.061377,0.510517,0.343673,0.355628,0.269313,0.162122,0.475838,0.581641,...,0.498596,0.188270,0.632206,0.000000,0.477054,0.236385,0.366821,0.417692,0.355217,0.211588
4,0.044695,0.088366,0.079901,0.380044,0.191186,0.285930,0.174621,0.173696,0.340225,0.502268,...,0.463240,0.051172,0.524663,0.000000,0.439538,0.207207,0.541598,0.422831,0.263589,0.114003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0.071849,0.052224,0.081633,0.394629,0.275060,0.323383,0.189943,0.213020,0.447939,0.497545,...,0.563792,0.071849,0.430305,0.000000,0.443608,0.223343,0.433582,0.418949,0.323383,0.168982
67,0.112611,0.085876,0.079028,0.430966,0.335291,0.307634,0.267841,0.198825,0.396926,0.465676,...,0.475104,0.108356,0.611999,0.000000,0.467881,0.235161,0.407623,0.346898,0.283508,0.174621
68,0.103232,0.131622,0.136466,0.505653,0.215142,0.403416,0.290869,0.210505,0.432361,0.578153,...,0.410961,0.079028,0.590601,0.000000,0.496486,0.277622,0.484317,0.425079,0.332601,0.153875
69,0.173696,0.062360,0.094017,0.466133,0.201158,0.340225,0.194853,0.163619,0.418859,0.426019,...,0.535348,0.119444,0.562944,0.015543,0.448233,0.228890,0.497966,0.385514,0.319947,0.225324


In [ ]:
# # 读取制表符分隔的CSV文件
# Methylation_val = pd.read_csv('../../DATA/THCA_parting/methylation_val.csv')
# # 显示 DataFrame
# Methylation_val

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.141733,0.106910,0.153344,0.493644,0.257636,0.364777,0.219981,0.252259,0.440933,0.539385,...,0.582180,0.122747,0.552270,0.000000,0.472881,0.277853,0.450420,0.411714,0.346607,0.226955
1,0.028390,0.000000,0.025907,0.412089,0.339159,0.326409,0.222676,0.185724,0.374486,0.467301,...,0.458928,0.080770,0.555985,0.000000,0.453428,0.246680,0.334347,0.376282,0.275763,0.107634
2,0.174621,0.119444,0.144015,0.472943,0.271975,0.361000,0.229209,0.221335,0.429724,0.526628,...,0.536246,0.159083,0.623637,0.000000,0.497386,0.282840,0.459338,0.405583,0.337160,0.222342
3,0.090002,0.090814,0.120774,0.400915,0.336384,0.306688,0.187426,0.226630,0.313737,0.474120,...,0.463836,0.081633,0.578060,0.000000,0.501140,0.267594,0.528654,0.419395,0.273644,0.161116
4,0.076376,0.142307,0.124047,0.425849,0.137060,0.304390,0.171828,0.135269,0.431048,0.441856,...,0.446534,0.081633,0.564433,0.051172,0.414318,0.232996,0.479574,0.380276,0.285492,0.148478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.094807,0.082492,0.042472,0.443077,0.211947,0.317140,0.168021,0.188690,0.397753,0.453853,...,0.503894,0.108356,0.410110,0.007266,0.416697,0.219640,0.508766,0.398576,0.322018,0.154404
66,0.132236,0.106181,0.060387,0.449694,0.173696,0.311363,0.216193,0.189526,0.367328,0.497913,...,0.474982,0.116748,0.634347,0.018202,0.449184,0.206464,0.509352,0.358338,0.268579,0.142307
67,0.094017,0.109788,0.089186,0.492175,0.180054,0.393041,0.249496,0.314460,0.467042,0.501088,...,0.578979,0.117427,0.656068,0.034420,0.463372,0.264348,0.453144,0.342183,0.305735,0.196056
68,0.052224,0.114694,0.053269,0.441395,0.206836,0.331480,0.254972,0.185724,0.399495,0.506450,...,0.478019,0.106181,0.626859,0.000000,0.445341,0.184434,0.417602,0.375327,0.273169,0.100981


In [ ]:
# 读取制表符分隔的CSV文件
miRNASeq_train = pd.read_csv('../../DATA/THCA_parting/3_tr.csv')
# 显示 DataFrame
miRNASeq_train

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.536138,0.493898,0.313622,0.673057,0.347664,0.882575,0.525694,0.302910,0.463281,0.524695,...,0.571598,0.190252,0.610287,0.434482,0.640451,0.505441,0.577362,0.312103,0.392741,0.308848
1,0.681927,0.643106,0.333751,0.773326,0.416641,0.874332,0.567864,0.436028,0.528897,0.615744,...,0.605867,0.159838,0.734076,0.591776,0.711390,0.622655,0.512113,0.265614,0.460598,0.404959
2,0.526849,0.501926,0.268437,0.636161,0.338596,0.826629,0.540336,0.320623,0.375533,0.525848,...,0.587036,0.203297,0.570025,0.499156,0.581459,0.512106,0.524621,0.310605,0.321102,0.327286
3,0.622823,0.551429,0.347272,0.742341,0.392350,0.912176,0.501390,0.408903,0.493463,0.573354,...,0.663959,0.269806,0.674698,0.533805,0.692656,0.477031,0.499395,0.371578,0.453525,0.378615
4,0.579327,0.519563,0.284819,0.687902,0.328320,0.868325,0.607898,0.362169,0.443423,0.509004,...,0.654633,0.223976,0.558896,0.528972,0.682052,0.480867,0.498257,0.361563,0.385183,0.296327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,0.574634,0.572532,0.303439,0.702631,0.386823,0.849504,0.595399,0.400728,0.454928,0.526443,...,0.684901,0.271815,0.602635,0.529764,0.663177,0.605549,0.563902,0.326938,0.415784,0.337581
206,0.578494,0.538260,0.273763,0.716966,0.354288,0.858739,0.565641,0.381254,0.461594,0.534191,...,0.609258,0.221354,0.628558,0.499733,0.649465,0.487716,0.544537,0.328057,0.385530,0.295312
207,0.613273,0.533456,0.339734,0.667563,0.309254,0.833195,0.554292,0.313353,0.416181,0.492470,...,0.554084,0.195447,0.628684,0.513073,0.638695,0.499231,0.515983,0.235825,0.358591,0.298488
208,0.535862,0.655576,0.248294,0.714125,0.369420,0.905228,0.588485,0.369132,0.425094,0.527233,...,0.660287,0.227530,0.652750,0.515958,0.646835,0.488626,0.510308,0.312804,0.407930,0.341037


In [ ]:
# 读取制表符分隔的CSV文件
miRNASeq_test = pd.read_csv('../../DATA/THCA_parting/3_te.csv')
# 显示 DataFrame
miRNASeq_test

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.636477,0.582610,0.311742,0.686883,0.363409,0.916336,0.598365,0.338654,0.397808,0.542641,...,0.652009,0.149287,0.642376,0.550381,0.706561,0.538832,0.509110,0.312026,0.463783,0.329504
1,0.537868,0.560139,0.293288,0.674952,0.355516,0.865736,0.629423,0.340724,0.436495,0.522952,...,0.588638,0.127681,0.611870,0.508268,0.615549,0.526583,0.558372,0.299117,0.370165,0.340715
2,0.588184,0.563345,0.275554,0.730194,0.371459,0.935580,0.592646,0.362311,0.479582,0.568918,...,0.649749,0.233179,0.636508,0.517827,0.655792,0.561696,0.541594,0.312743,0.431669,0.380083
3,0.647425,0.574234,0.350328,0.741731,0.375318,0.831933,0.586606,0.365600,0.413820,0.543183,...,0.581911,0.161881,0.635631,0.546277,0.692837,0.494645,0.508547,0.279187,0.443970,0.337727
4,0.560996,0.547512,0.278014,0.737094,0.351183,0.917013,0.587173,0.380527,0.476647,0.544561,...,0.554261,0.210073,0.669128,0.539841,0.652562,0.494985,0.505819,0.311685,0.420646,0.344907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0.572572,0.535556,0.312320,0.688941,0.332178,0.875345,0.565469,0.389930,0.432463,0.529903,...,0.604858,0.147871,0.609722,0.504016,0.656599,0.434673,0.479017,0.282312,0.401255,0.334039
67,0.570569,0.495804,0.378278,0.740705,0.337366,0.857559,0.628539,0.322244,0.425449,0.485416,...,0.613666,0.252531,0.609251,0.499607,0.639922,0.493636,0.508596,0.326887,0.368404,0.290804
68,0.596292,0.529825,0.333195,0.675292,0.336263,0.893040,0.554459,0.340824,0.503864,0.526884,...,0.597962,0.249464,0.646282,0.452249,0.659332,0.464961,0.562734,0.326897,0.368100,0.310360
69,0.575455,0.464524,0.308397,0.669859,0.340807,0.862595,0.590233,0.303528,0.437918,0.529272,...,0.648281,0.170990,0.551651,0.497183,0.635714,0.465937,0.561572,0.405034,0.320404,0.297863


In [ ]:
# # 读取制表符分隔的CSV文件
# miRNASeq_val = pd.read_csv('../../DATA/THCA_parting/miRNASeq_val.csv')
# # 显示 DataFrame
# miRNASeq_val

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.552139,0.559067,0.285569,0.718629,0.390478,0.833890,0.539042,0.403945,0.461322,0.546504,...,0.613052,0.203944,0.637093,0.555595,0.659333,0.533907,0.526485,0.326518,0.441615,0.345450
1,0.580002,0.529317,0.329242,0.738768,0.388183,0.904719,0.580431,0.392392,0.485232,0.548183,...,0.686109,0.231781,0.617823,0.544666,0.661086,0.507549,0.544031,0.303962,0.447308,0.336100
2,0.561408,0.513636,0.315951,0.673154,0.318711,0.922501,0.553679,0.307780,0.455743,0.537512,...,0.580173,0.162286,0.585571,0.492083,0.634090,0.512798,0.561921,0.270619,0.345588,0.301247
3,0.549919,0.462672,0.340425,0.611309,0.316472,0.861999,0.563730,0.292498,0.400651,0.465073,...,0.590594,0.206259,0.554519,0.459943,0.650416,0.446624,0.499137,0.285911,0.324034,0.295996
4,0.567981,0.555092,0.297521,0.694480,0.385651,0.943919,0.565570,0.351122,0.397546,0.525114,...,0.621068,0.232070,0.579691,0.507443,0.663793,0.517530,0.549106,0.347407,0.421452,0.346779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.580880,0.497651,0.258057,0.701615,0.336419,0.864107,0.596609,0.327971,0.499936,0.541747,...,0.639236,0.218292,0.619704,0.499202,0.675908,0.486819,0.553745,0.334531,0.430577,0.311705
66,0.604719,0.491897,0.274434,0.656380,0.357742,0.839752,0.502004,0.357297,0.431040,0.520637,...,0.620938,0.178357,0.582983,0.498561,0.620178,0.492380,0.497586,0.306908,0.383063,0.311347
67,0.600014,0.562987,0.249852,0.701769,0.402879,0.927926,0.572836,0.419111,0.478940,0.564210,...,0.632291,0.264023,0.658148,0.526569,0.683633,0.515043,0.483666,0.328925,0.417604,0.359054
68,0.580941,0.550428,0.367881,0.691000,0.394067,0.853389,0.531384,0.400435,0.448872,0.559928,...,0.623368,0.207858,0.662403,0.559818,0.682088,0.512996,0.504750,0.343361,0.445321,0.356560


In [ ]:
# 读取制表符分隔的CSV文件
RNAseq_train = pd.read_csv('../../DATA/THCA_parting/2_tr.csv')
# 显示 DataFrame
RNAseq_train

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.798907,0.735724,0.751890,0.831284,0.724953,0.473487,0.944846,0.379073,0.203647,0.685920,...,0.385380,0.292930,0.234979,0.260191,0.253372,0.234694,0.155948,0.299638,0.314865,0.263381
1,0.807236,0.712284,0.730247,0.819646,0.734358,0.477053,0.941102,0.395230,0.284018,0.682869,...,0.365879,0.275455,0.361636,0.284818,0.388235,0.239268,0.352297,0.400933,0.435073,0.266057
2,0.816905,0.724565,0.719784,0.837038,0.748833,0.480831,0.954793,0.365776,0.286031,0.700446,...,0.307314,0.284587,0.250329,0.182719,0.283641,0.233600,0.281007,0.331042,0.377167,0.317765
3,0.793171,0.737116,0.759549,0.841422,0.733765,0.517666,0.919615,0.391978,0.255071,0.666125,...,0.403233,0.223435,0.262910,0.216989,0.191927,0.338639,0.178280,0.252927,0.341936,0.347774
4,0.805424,0.722645,0.767946,0.800873,0.729937,0.502907,0.942959,0.388526,0.260258,0.668824,...,0.351525,0.285677,0.318781,0.271669,0.171380,0.244158,0.271588,0.271920,0.356414,0.345516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,0.797760,0.737862,0.763707,0.834947,0.716293,0.479883,0.934261,0.425524,0.228912,0.674421,...,0.217326,0.242386,0.208652,0.291812,0.250175,0.261176,0.212059,0.305331,0.330686,0.343881
206,0.807673,0.737630,0.761739,0.830347,0.740090,0.538156,0.941689,0.347578,0.239740,0.675897,...,0.299436,0.245941,0.235716,0.259349,0.228823,0.277467,0.282033,0.316850,0.378633,0.268288
207,0.820332,0.733834,0.758290,0.813778,0.721499,0.445163,0.926757,0.344475,0.138328,0.667812,...,0.401012,0.247518,0.239063,0.216090,0.224356,0.351130,0.230011,0.351508,0.349275,0.376025
208,0.840917,0.725794,0.765166,0.818968,0.732587,0.496392,0.951199,0.361329,0.236333,0.681058,...,0.279500,0.294944,0.197405,0.290212,0.254256,0.213738,0.268842,0.312756,0.389861,0.287308


In [ ]:
# 读取制表符分隔的CSV文件
RNAseq_test = pd.read_csv('../../DATA/THCA_parting/2_te.csv')
# 显示 DataFrame
RNAseq_test

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.793003,0.709606,0.721879,0.834684,0.753937,0.467375,0.960719,0.352485,0.219769,0.693918,...,0.371620,0.211666,0.277812,0.214386,0.233559,0.354043,0.266821,0.311767,0.400873,0.309220
1,0.811867,0.742295,0.762346,0.846719,0.736210,0.549367,0.940407,0.360581,0.267450,0.695093,...,0.349109,0.240599,0.269812,0.269671,0.237568,0.217153,0.239773,0.314512,0.328268,0.335625
2,0.806733,0.710420,0.741262,0.843361,0.732442,0.531331,0.942780,0.409817,0.288463,0.683201,...,0.198458,0.228993,0.220373,0.265692,0.266537,0.231439,0.163615,0.293377,0.305227,0.351944
3,0.814385,0.718509,0.754616,0.839521,0.738961,0.526235,0.943525,0.392392,0.319183,0.697681,...,0.257104,0.260821,0.288695,0.263305,0.332982,0.229277,0.245703,0.327600,0.355241,0.322626
4,0.815538,0.727702,0.745215,0.829479,0.738741,0.504837,0.947324,0.366882,0.167566,0.688812,...,0.371046,0.264504,0.263607,0.316706,0.245949,0.246698,0.286809,0.314163,0.347649,0.276571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0.804041,0.699481,0.708341,0.833917,0.734879,0.539785,0.940506,0.407475,0.317361,0.702691,...,0.124130,0.362786,0.315348,0.303023,0.231527,0.192564,0.323248,0.410385,0.418990,0.263463
67,0.828682,0.723210,0.750118,0.848689,0.758602,0.500481,0.949241,0.356482,0.234229,0.687243,...,0.343994,0.243092,0.292688,0.249129,0.280585,0.284343,0.279401,0.343331,0.337790,0.233913
68,0.820888,0.726444,0.744190,0.827705,0.734209,0.474903,0.942679,0.347207,0.246817,0.683196,...,0.323588,0.226388,0.287337,0.258655,0.232294,0.283436,0.257268,0.321768,0.374917,0.289345
69,0.808246,0.759955,0.747215,0.824923,0.719057,0.441630,0.924886,0.404335,0.293269,0.665009,...,0.368334,0.224143,0.320211,0.258251,0.193026,0.262592,0.221907,0.313015,0.320002,0.387227


In [ ]:
# # 读取制表符分隔的CSV文件
# RNAseq_val = pd.read_csv('../../DATA/THCA_parting/RNAseq_val.csv')
# # 显示 DataFrame
# RNAseq_val

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.822696,0.749512,0.738373,0.829757,0.725952,0.510500,0.928945,0.401692,0.279266,0.673976,...,0.337899,0.256514,0.289538,0.182423,0.265771,0.254107,0.222156,0.348486,0.386689,0.317780
1,0.807597,0.717130,0.739479,0.836040,0.743131,0.509932,0.938481,0.344084,0.228623,0.689352,...,0.273958,0.292204,0.217053,0.243807,0.287193,0.272302,0.274035,0.366102,0.316983,0.308586
2,0.808222,0.711237,0.751831,0.856981,0.768841,0.529645,0.954614,0.372392,0.335992,0.727246,...,0.379027,0.326835,0.305716,0.280504,0.323942,0.273627,0.195085,0.405561,0.359562,0.338459
3,0.807853,0.739236,0.756555,0.838984,0.749880,0.512322,0.946342,0.361297,0.228889,0.679450,...,0.357983,0.246972,0.261173,0.204835,0.185487,0.220200,0.183395,0.291305,0.310701,0.277985
4,0.807047,0.729387,0.763370,0.818535,0.726867,0.520051,0.940714,0.412594,0.297168,0.677224,...,0.402075,0.205109,0.222423,0.211773,0.076387,0.233315,0.283959,0.347730,0.391543,0.266929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.815769,0.725591,0.742734,0.825777,0.741116,0.514395,0.954288,0.362880,0.302591,0.675439,...,0.350479,0.243489,0.283342,0.271770,0.268658,0.197210,0.208597,0.324356,0.399081,0.329099
66,0.817828,0.735251,0.729180,0.812079,0.723800,0.484208,0.912951,0.364798,0.114950,0.676715,...,0.295223,0.346479,0.293446,0.316083,0.347091,0.259914,0.304735,0.333377,0.409943,0.248880
67,0.815332,0.737245,0.754298,0.844369,0.749922,0.552492,0.947020,0.364876,0.255961,0.707385,...,0.329426,0.262290,0.244829,0.186200,0.235827,0.133048,0.184496,0.297047,0.374515,0.217278
68,0.795091,0.698683,0.734965,0.843130,0.741016,0.537043,0.938839,0.355818,0.268145,0.689790,...,0.291312,0.200119,0.177841,0.217369,0.306494,0.246859,0.316634,0.321032,0.332343,0.248622


In [ ]:
# 使用np.load()函数加载.npy文件
labels_train = pd.read_csv('../../DATA/THCA_parting/labels_tr.csv')
labels_train.value_counts()

0  
1.0    113
0.0     97
Name: count, dtype: int64

In [ ]:
# 使用np.load()函数加载.npy文件
labels_test = pd.read_csv('../../DATA/THCA_parting/labels_te.csv')
labels_test.value_counts()

0  
0.0    36
1.0    35
Name: count, dtype: int64

In [ ]:
# # 使用np.load()函数加载.npy文件
# labels_val = pd.read_csv('../../DATA/THCA_parting/labels_val.csv')
# labels_val.value_counts()

0  
0.0    36
1.0    34
Name: count, dtype: int64

In [17]:
label_dim = len(np.unique(labels_train))
label_dim

2

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用 GPU 

In [ ]:
# 调用函数，将这些 numpy 数组转换为 tensors
train_tensors = batch_dataframe_to_tensors(Methylation_train, miRNASeq_train, RNAseq_train)

# 现在 tensors 是一个包含这些转换后张量的列表
Methylation_train = train_tensors[0]
miRNASeq_train = train_tensors[1]
RNAseq_train = train_tensors[2]

DataFrame 0 已转换为 Tensor，并移动到 cuda:0
DataFrame 1 已转换为 Tensor，并移动到 cuda:0
DataFrame 2 已转换为 Tensor，并移动到 cuda:0


In [ ]:
# 调用函数，将这些 numpy 数组转换为 tensors
test_tensors = batch_dataframe_to_tensors(Methylation_test, miRNASeq_test, RNAseq_test)

# 现在 tensors 是一个包含这些转换后张量的列表
Methylation_test = test_tensors[0]
miRNASeq_test = test_tensors[1]
RNAseq_test = test_tensors[2]

DataFrame 0 已转换为 Tensor，并移动到 cuda:0
DataFrame 1 已转换为 Tensor，并移动到 cuda:0
DataFrame 2 已转换为 Tensor，并移动到 cuda:0


In [ ]:
# # 调用函数，将这些 numpy 数组转换为 tensors
# val_tensors = batch_dataframe_to_tensors(Methylation_val, miRNASeq_val, RNAseq_val)

# # 现在 tensors 是一个包含这些转换后张量的列表
# Methylation_val = val_tensors[0]
# miRNASeq_val = val_tensors[1]
# RNAseq_val = val_tensors[2]

DataFrame 0 已转换为 Tensor，并移动到 cuda:0
DataFrame 1 已转换为 Tensor，并移动到 cuda:0
DataFrame 2 已转换为 Tensor，并移动到 cuda:0


In [22]:
# 将DataFrame的值转换为Tensor
labels_tr_tensor = torch.LongTensor(labels_train.values)
labels_tr_tensor

tensor([[0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],

In [23]:
onehot_labels_tr_tensor = one_hot_tensor(labels_tr_tensor, num_class)

In [24]:
# 将DataFrame的值转换为Tensor
labels_te_tensor = torch.LongTensor(labels_test.values)
labels_te_tensor

tensor([[0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0]])

In [25]:
# 将DataFrame的值转换为Tensor
labels_tr_tensor = torch.LongTensor(labels_train.values)
labels_tr_tensor

tensor([[0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],

In [ ]:
# # 将DataFrame的值转换为Tensor
# labels_val_tensor = torch.LongTensor(labels_val.values)
# labels_val_tensor

tensor([[1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1]])

In [ ]:
# 定义数据集
train_dataset = TensorDataset(Methylation_train, miRNASeq_train, RNAseq_train, labels_tr_tensor)
test_dataset = TensorDataset(Methylation_test, miRNASeq_test, RNAseq_test, labels_te_tensor)
# val_dataset = TensorDataset(Methylation_val, miRNASeq_val, RNAseq_val, labels_val_tensor)

# 定义 DataLoader，使用自定义采样器
train_loader = DataLoader(train_dataset, batch_size=batch_size, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)  # 测试集一般不需要打乱顺序
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)  # 验证集也一般不需要重复采样

In [29]:
for i, batch in enumerate(train_loader):
    print(f"Batch {i + 1}: {len(batch[0])} samples")


Batch 1: 32 samples
Batch 2: 32 samples
Batch 3: 32 samples
Batch 4: 32 samples
Batch 5: 32 samples
Batch 6: 32 samples
Batch 7: 18 samples


In [30]:
for i, batch in enumerate(test_loader):
    print(f"Batch {i + 1}: {len(batch[0])} samples")

Batch 1: 32 samples
Batch 2: 32 samples
Batch 3: 7 samples


In [ ]:
# for i, batch in enumerate(val_loader):
#     print(f"Batch {i + 1}: {len(batch[0])} samples")

Batch 1: 32 samples
Batch 2: 32 samples
Batch 3: 6 samples


In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib import rcParams
from typing import Dict, Tuple, List
import gc

def plot_metrics(lr: float, depth: int, heads: int, dim_head: int, beta: float, adj_parameter: int, dropout: float) -> None:
    """生成四分类训练指标图表"""
    
    # ================== 配置参数 ==================
    SAVE_DIR = '../../result/THCA_parting/'
    FILE_FORMAT = 'pdf'
    DPI = 600
    FONT_CONFIG = {
        'font.family': 'DejaVu Sans',
        'font.size': 12,
        'axes.titlesize': 14,
        'axes.labelsize': 12,
        'legend.fontsize': 10
    }
    STYLE_CONFIG = {
        'train': {'color': '#E74C3C', 'ls': '-', 'lw': 2.5, 'alpha': 0.9},
        'val': {'color': '#3498DB', 'ls': '-', 'lw': 2.5, 'alpha': 0.9},
        'test': {'color': '#2ECC71', 'ls': '-', 'lw': 2.5, 'alpha': 0.9}
    }
    
    # ================== 初始化设置 ==================
    rcParams.update(FONT_CONFIG)
    os.makedirs(SAVE_DIR, exist_ok=True)
    param_id = f"lr{lr:.0e}_depth{depth}heads{heads}dim_head{dim_head}adj_parameter{adj_parameter}dropout{dropout}beta{beta}".replace('.', '').replace('+', '')

    try:
        # ================== 创建画布 ==================
        fig = plt.figure(figsize=(18, 12), dpi=DPI, facecolor='white')
        fig.suptitle(f'Training Metrics ({param_id})', y=1.02, fontsize=16, fontweight='bold')
        
        # ================== 子图配置 ==================
        axes = [
            fig.add_subplot(221),  # Loss
            fig.add_subplot(222),  # Accuracy
            fig.add_subplot(223),  # F1 Weighted
            fig.add_subplot(224)   # F1 Macro
        ]
        
        metric_data = [
            ('Loss', (train_losses, test_losses), 'log' if max(train_losses) > 1e3 else 'linear'),
            ('Accuracy', (train_accuracies, test_accuracies), 'linear'),
            ('F1 Weighted', (train_f1_weighted_scores, test_f1_weighted_scores), 'linear'),
            ('F1 Macro', (train_f1_macro_scores, test_f1_macro_scores), 'linear')
        ]

        # ================== 绘制图表 ==================
        for ax, (title, (train, test), scale) in zip(axes, metric_data):
            ax.plot(train, label='Train', **STYLE_CONFIG['train'])
            # ax.plot(val, label='Val', **STYLE_CONFIG['val'])
            ax.plot(test, label='Test', **STYLE_CONFIG['test'])
            
            ax.set_title(title, pad=15, fontweight='semibold')
            ax.set_xlabel('Epochs', labelpad=10)
            ax.set_ylabel(title, labelpad=10)
            ax.grid(True, which='both', ls='--', alpha=0.4)
            ax.set_yscale(scale)
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            
            if ax == axes[0]:
                handles, labels = ax.get_legend_handles_labels()
                fig.legend(handles, labels, loc='upper center', 
                         bbox_to_anchor=(0.5, 1.0), ncol=3, frameon=False)

        # ================== 布局优化 ==================
        plt.tight_layout(pad=3.0, w_pad=2.0, h_pad=2.0)
        
        # ================== 保存图像 ==================
        save_path = os.path.join(SAVE_DIR, f'metrics_{param_id}.{FILE_FORMAT}')
        plt.savefig(
            save_path,
            dpi=DPI,
            bbox_inches='tight',
            facecolor=fig.get_facecolor(),
            edgecolor='none',
            transparent=False,
            metadata={'CreationDate': None}
        )
        
    except Exception as e:
        print(f"图表生成失败: {str(e)}")
    finally:
        plt.close(fig)
        gc.collect()

In [ ]:
def train_epoch(data_loader, model, optimizer, adj_parameter, output_attentions=False):
    model.train()
    total_loss = 0.0
    all_probs, all_labels = [], []

    for batch in data_loader:
        Methylation_batch, miRNASeq_batch, RNAseq_batch, labels_batch = [
            tensor.to(device) for tensor in batch
        ]

        # 生成三维邻接张量
        graph_train = np.stack([
            gen_trte_adj_mat(Methylation_batch.cpu(), adj_parameter),
            gen_trte_adj_mat(miRNASeq_batch.cpu(), adj_parameter),
            gen_trte_adj_mat(RNAseq_batch.cpu(), adj_parameter)
        ], axis=-1)
        graph_train = torch.from_numpy(graph_train).float().to(device).permute(0, 2, 1)

        # 梯度更新流程
        optimizer.zero_grad()
        ci_loss, ci, feature = model(
            Methylation_batch, 
            miRNASeq_batch,
            RNAseq_batch,
            graph_train,
            labels_batch,
            output_attentions=output_attentions
        )
        ci_loss.backward()
        optimizer.step()

        # 结果收集
        total_loss += ci_loss.item() * labels_batch.size(0)
        all_probs.append(F.softmax(ci, dim=1).detach().cpu().numpy())
        all_labels.append(labels_batch.detach().cpu().numpy().flatten())

    return (
        total_loss / len(data_loader.dataset),
        np.concatenate(all_probs),
        np.concatenate(all_labels)
    )

In [ ]:
# def val_epoch(data_loader, model, adj_parameter, output_attentions=False):
#     model.eval()
#     total_loss = 0.0
#     all_labels, all_probs = [], []

#     with torch.no_grad():
#         for batch in data_loader:
#             # 数据预处理
#             Methylation_batch, miRNASeq_batch, RNAseq_batch, labels_batch = [
#                 x.to(device) for x in batch
#             ]
#             # print(miRNASeq_batch.shape) #torch.Size([32, 217])
#             # 生成三维邻接张量
#             graph_val = torch.from_numpy(np.stack([
#                 gen_trte_adj_mat(d.cpu(), adj_parameter) 
#                 for d in [Methylation_batch, miRNASeq_batch, RNAseq_batch]
#             ], axis=-1)).float().to(device).permute(0,2,1)

#             # 前向计算
#             ci_loss, ci = model(
#                 Methylation_batch, 
#                 miRNASeq_batch,
#                 RNAseq_batch,
#                 graph_val,
#                 labels_batch,
#                 output_attentions=output_attentions
#             )

#             # 累计结果
#             total_loss += ci_loss.item() * labels_batch.size(0)
#             all_probs.append(F.softmax(ci, 1).detach().cpu().numpy())
#             all_labels.append(labels_batch.detach().cpu().numpy().flatten())

#     return (
#         total_loss / len(data_loader.dataset),
#         np.concatenate(all_probs),
#         np.concatenate(all_labels)
#     )

In [ ]:
def test_epoch(data_loader, model, adj_parameter, output_attentions=False):
    model.eval()
    total_loss = 0.0
    all_probs, all_labels = [], []

    with torch.no_grad():
        for batch in data_loader:
            # 数据预处理 (保持原始变量名)
            Methylation_batch, miRNASeq_batch, RNAseq_batch, labels_test_tensor = [
                tensor.to(device) for tensor in batch
            ]

            # 生成邻接矩阵 (保持原始逻辑但更紧凑)
            graph_test = torch.from_numpy(
                np.stack([
                    gen_trte_adj_mat(Methylation_batch.cpu(), adj_parameter),
                    gen_trte_adj_mat(miRNASeq_batch.cpu(), adj_parameter),
                    gen_trte_adj_mat(RNAseq_batch.cpu(), adj_parameter)
                ], axis=-1)
            ).float().to(device).permute(0, 2, 1)

            # 前向计算
            loss, logits, feature = model(
                Methylation_batch,
                miRNASeq_batch,
                RNAseq_batch,
                graph_test,
                labels_test_tensor,
                output_attentions=output_attentions
            )
            feature = pd.DataFrame(feature.detach().cpu().numpy())

            # 保存为 CSV 文件
            feature.to_csv('feature.csv', index=False)
            # 结果收集
            total_loss += loss.item() * labels_test_tensor.size(0)
            all_probs.append(F.softmax(logits, dim=1).detach().cpu().numpy())
            labels_test_tensor_df = pd.DataFrame(labels_test_tensor.detach().cpu().numpy())
            labels_test_tensor_df.to_csv('all_probs.csv', index=False)
            all_labels.append(labels_test_tensor.detach().cpu().numpy().flatten())

    return (
        total_loss / len(data_loader.dataset),
        np.concatenate(all_probs),
        np.concatenate(all_labels)
    )

In [ ]:
# 用于保存指标数据的列表（移除了AUC相关指标）
train_losses = []
val_losses = []
test_losses = []
val_accuracies = []
val_f1_weighted_scores = []
val_f1_macro_scores = []
test_accuracies = []
test_f1_weighted_scores = []
test_f1_macro_scores = []
train_accuracies = []
train_f1_weighted_scores = []
train_f1_macro_scores = []

def train_test(
        Methylation_train,
        miRNASeq_train,
        RNAseq_train,
        lr,
        depth, 
        heads, 
        dim_head,
        beta, 
        attn_dropout,
        ff_dropout, 
        classifier_dropout,
        adj_parameter,
        num_epoch_pretrain
):
    # 初始化最佳指标跟踪器（以val_f1_macro为选择标准）
    best_metrics = {
        'epoch': -1,
        # 'val_f1_macro': -1.0,
        'test_acc': 0.0,
        'test_f1_weighted': 0.0,
        'test_f1_macro': 0.0,
    }
    
    DEPTH = depth
    HEAD = heads
    HEAD_DIM = dim_head

    classifier_input = [Methylation_train.shape[1], miRNASeq_train.shape[1], RNAseq_train.shape[1]]
    classifier_dim = [
        [classifier_input[0], 300, 200],
        [classifier_input[1], 300, 200],
        [classifier_input[2], 300, 200]
    ]
    col_dim = 32

    # 确定 row_dim 和 embeding_num
    if num_view <= 2:
        embeding = True
        embeding_num = 32
        row_dim = embeding_num
    else:
        embeding = False
        row_dim = num_view
        embeding_num = 32
    
    model = pathformer_model(
        row_dim=row_dim,
        col_dim=col_dim,
        depth=DEPTH,
        heads=HEAD,
        dim_head=HEAD_DIM,
        classifier_input=classifier_input,
        classifier_dim=classifier_dim,
        label_dim=label_dim,
        embeding=embeding,
        embeding_num=embeding_num,
        beta=beta,
        attn_dropout=attn_dropout,
        ff_dropout=ff_dropout,
        classifier_dropout=classifier_dropout,
        num_view=num_view,
        dim_hvcdn=dim_hvcdn
    )

    if torch.cuda.is_available():
        model.to(device)

    logging.info("\nPretrain GCNs...")
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    logging.info("\nTraining...")
    for epoch in range(num_epoch_pretrain + 1):
        # 训练阶段
        train_loss, train_probs, labels_train_tensor = train_epoch(train_loader, model, optimizer, adj_parameter, output_attentions=False)
        train_acc = accuracy_score(labels_train_tensor, train_probs.argmax(1))
        train_f1_weighted = f1_score(labels_train_tensor, train_probs.argmax(1), average='weighted')
        train_f1_macro = f1_score(labels_train_tensor, train_probs.argmax(1), average='macro')

        logging.info(f"Train Epoch {epoch}, Loss: {train_loss}")
        logging.info(f"Train Epoch {epoch}, ACC: {train_acc:.3f}")
        logging.info(f"Train Epoch {epoch}, F1 Weighted: {train_f1_weighted:.3f}")
        logging.info(f"Train Epoch {epoch}, F1 Macro: {train_f1_macro:.3f}")
        
        # 验证阶段
        # val_loss, val_probs, labels_validation_tensor = val_epoch(val_loader, model, adj_parameter, output_attentions=False)
        # val_acc = accuracy_score(labels_validation_tensor, val_probs.argmax(1))
        # val_f1_weighted = f1_score(labels_validation_tensor, val_probs.argmax(1), average='weighted')
        # val_f1_macro = f1_score(labels_validation_tensor, val_probs.argmax(1), average='macro')
        
        # logging.info(f"Val Epoch {epoch}, Loss: {val_loss}")
        # logging.info(f"Val Epoch {epoch}, ACC: {val_acc:.3f}")
        # logging.info(f"Val Epoch {epoch}, F1 Weighted: {val_f1_weighted:.3f}")
        # logging.info(f"Val Epoch {epoch}, F1 Macro: {val_f1_macro:.3f}")
        
        # 测试阶段
        test_loss, test_probs, labels_test_tensor = test_epoch(test_loader, model, adj_parameter, output_attentions=False)
        test_acc = accuracy_score(labels_test_tensor, test_probs.argmax(1))
        test_f1_weighted = f1_score(labels_test_tensor, test_probs.argmax(1), average='weighted')
        test_f1_macro = f1_score(labels_test_tensor, test_probs.argmax(1), average='macro')
        
        logging.info(f"Test Epoch {epoch}, Loss: {test_loss}")
        logging.info(f"Test Epoch {epoch}, ACC: {test_acc:.3f}")
        logging.info(f"Test Epoch {epoch}, F1 Weighted: {test_f1_weighted:.3f}")
        logging.info(f"Test Epoch {epoch}, F1 Macro: {test_f1_macro:.3f}")

        # 更新最佳指标（以val_f1_macro为选择标准）
        if test_f1_weighted > best_metrics['test_f1_weighted'] or \
           (test_f1_weighted == best_metrics['test_f1_weighted'] and test_f1_macro > best_metrics['test_f1_macro']):
            best_metrics.update({
                'epoch': epoch,
                # 'val_f1_macro': val_f1_macro,
                'test_acc': test_acc,
                'test_f1_weighted': test_f1_weighted,
                'test_f1_macro': test_f1_macro
            })
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"Saved best model at epoch {epoch}")

        logging.info("\n")

        # 保存训练指标
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        train_f1_weighted_scores.append(train_f1_weighted)
        train_f1_macro_scores.append(train_f1_macro)
        
        # val_losses.append(val_loss)
        # val_accuracies.append(val_acc)
        # val_f1_weighted_scores.append(val_f1_weighted)
        # val_f1_macro_scores.append(val_f1_macro)
        
        test_losses.append(test_loss)
        test_accuracies.append(test_acc)
        test_f1_weighted_scores.append(test_f1_weighted)
        test_f1_macro_scores.append(test_f1_macro)

        # 绘制并保存指标图（需调整绘图函数）
        # plot_metrics(lr, depth, heads, dim_head, beta, adj_parameter, dropout=attn_dropout)

    return (
        best_metrics['test_acc'],
        best_metrics['test_f1_weighted'],
        best_metrics['test_f1_macro']
    )

In [ ]:
# # 用于保存指标数据的列表
# train_losses = []
# val_losses = []
# test_losses = []
# val_accuracies = []
# val_f1_scores = []
# val_auc_scores = []
# test_accuracies = []
# test_f1_scores = []
# test_auc_scores = []
# train_accuracies = []
# train_f1_scores = []
# train_auc_scores = []
# train_c_index = []
# val_c_index = []
# test_c_index = []

# def train_test(
#         Methylation_train,
#         Methylation_test,
#         miRNASeq_train,
#         miRNASeq_test,
#         RNAseq_train,
#         RNAseq_test,
#         label_train,
#         label_test, 
#         labels_tr_tensor,
#         lr,
#         depth, 
#         heads, 
#         dim_head,
#         beta, 
#         attn_dropout,
#         ff_dropout, 
#         classifier_dropout,
#         adj_parameter,
#         num_epoch_pretrain
# ):
    
#     DEPTH = depth
#     HEAD = heads
#     HEAD_DIM = dim_head

#     classifier_input = [Methylation_train.shape[1], miRNASeq_train.shape[1], RNAseq_train.shape[1]]
#     classifier_dim = [
#         [classifier_input[0], 300, 200],
#         [classifier_input[1], 300, 200],
#         [classifier_input[2], 300, 200]
#     ]
#     col_dim = 71

#     # 确定 row_dim 和 embeding_num
#     if num_view <= 2:
#         embeding = True
#         embeding_num = 71
#         row_dim = embeding_num  # 请根据需要修改此处
#     else:
#         embeding = False
#         row_dim = num_view
#         embeding_num = 71
    
#     model = pathformer_model(
#         row_dim=row_dim,
#         col_dim=col_dim,
#         depth=DEPTH,
#         heads=HEAD,
#         dim_head=HEAD_DIM,
#         classifier_input=classifier_input,
#         classifier_dim=classifier_dim,
#         label_dim=label_dim,
#         embeding=embeding,
#         embeding_num=embeding_num,
#         beta=beta,
#         attn_dropout=attn_dropout,
#         ff_dropout=ff_dropout,
#         classifier_dropout=classifier_dropout,
#         num_view=num_view,
#         dim_hvcdn=dim_hvcdn
#     )

#     if torch.cuda.is_available():
#         model.to(device)

#     print("\nPretrain GCNs...")
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#     print("\nTraining...")
#     for epoch in range(num_epoch_pretrain + 1):
#         # 每一轮都进行训练
#         train_loss, train_probs, labels_train_tensor = train_epoch(train_loader, model, optimizer, adj_parameter, epoch, output_attentions=False)
#         train_acc = accuracy_score(labels_train_tensor, train_probs.argmax(1))
#         train_f1 = f1_score(labels_train_tensor, train_probs.argmax(1))
#         train_auc = roc_auc_score(labels_train_tensor, train_probs[:, 1])
# #         print(f"Train Epoch {epoch}, Loss: {train_loss}")
# #         print(f"Train Epoch {epoch}, ACC: Train ACC: {train_acc:.3f}")
# #         print(f"Train Epoch {epoch}, f1: Train F1: {train_f1:.3f}")
# #         print(f"Train Epoch {epoch}, AUC: Train AUC: {train_auc:.3f}")
        
#         # 每一轮都进行验证
#         val_loss, val_probs, labels_val_tensor = val_epoch(val_loader, model, adj_parameter, epoch, output_attentions=False)
#         val_acc = accuracy_score(labels_val_tensor, val_probs.argmax(1))
#         val_f1 = f1_score(labels_val_tensor, val_probs.argmax(1))
#         val_auc = roc_auc_score(labels_val_tensor, val_probs[:, 1])
# #         print(f"val Epoch {epoch}, Loss: {val_loss}")
# #         print(f"val Epoch {epoch}, ACC: val ACC: {val_acc:.3f}")
# #         print(f"val Epoch {epoch}, f1: val F1: {val_f1:.3f}")
# #         print(f"val Epoch {epoch}, AUC: val AUC: {val_auc:.3f}")
        
#         # 每一轮都进行测试
#         test_loss, test_probs, labels_test_tensor = test_epoch(test_loader, model, adj_parameter, epoch, output_attentions=False)
#         test_acc = accuracy_score(labels_test_tensor, test_probs.argmax(1))
#         test_f1 = f1_score(labels_test_tensor, test_probs.argmax(1))
#         test_auc = roc_auc_score(labels_test_tensor, test_probs[:, 1])
# #         print(f"test Epoch {epoch}, Loss: {test_loss}")
# #         print(f"test Epoch {epoch}, ACC: test ACC: {test_acc:.3f}")
# #         print(f"test Epoch {epoch}, f1: test F1: {test_f1:.3f}")
# #         print(f"test Epoch {epoch}, AUC: test AUC: {test_auc:.3f}")

#         print("\n")

#         # 保存训练指标
#         train_losses.append(train_loss)
#         train_accuracies.append(train_acc)
#         train_f1_scores.append(train_f1)
#         train_auc_scores.append(train_auc)
        
#         val_losses.append(val_loss)
#         val_accuracies.append(val_acc)
#         val_f1_scores.append(val_f1)
#         val_auc_scores.append(val_auc)
        
#         test_losses.append(test_loss)
#         test_accuracies.append(test_acc)
#         test_f1_scores.append(test_f1)
#         test_auc_scores.append(test_auc)

#         # 绘制并保存指标图
#         plot_metrics(lr, depth, heads, dim_head, beta, dropout=attn_dropout)

#     # 返回最终的预测结果和指标
#     final_predictions = test_probs.argmax(1)
#     return final_predictions, test_acc, test_f1, test_auc


In [ ]:
import logging
# import os
# import time
# import pandas as pd
# import torch
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# # 配置日志和路径
# LOG_FILE = '../../result/THCA_parting/training_log_CMFM.txt'
# RESULT_FILE = '../../result/THCA_parting/mogat_training_results.xlsx'
# os.makedirs(os.path.dirname(LOG_FILE), exist_ok=True)

# logging.basicConfig(
#     filename=LOG_FILE,
#     level=logging.INFO,
#     format='%(asctime)s - %(message)s',
#     filemode='w'  # 每次运行覆盖旧日志
# )

# # 超参数配置（便于维护和修改）
# HP_CONFIG = {
#     'lr': [0.0001, 0.001, 0.01],
#     'depth': [2, 3, 4, 6],
#     'heads': [4, 6, 8],
#     'dim_head': [12, 16, 32, 64],
#     'dropout': [0.3],
#     'beta': [1],
#     'adj_parameter': [2, 4, 6, 8, 10]
# }

# # 计算总组合数
# total_combinations = (
#     len(HP_CONFIG['lr']) *
#     len(HP_CONFIG['depth']) *
#     len(HP_CONFIG['heads']) *
#     len(HP_CONFIG['dim_head']) *
#     len(HP_CONFIG['dropout']) *
#     len(HP_CONFIG['beta']) *
#     len(HP_CONFIG['adj_parameter'])
# )

# # 结果缓存配置
# BATCH_SIZE = 5  # 每5个结果写入一次文件
# results_cache = []

# def save_results(batch: list, force_save: bool = False):
#     """批量保存结果到文件"""
#     if not force_save and len(batch) < BATCH_SIZE:
#         return
    
#     try:
#         df = pd.DataFrame(batch)
#         header = not os.path.exists(RESULT_FILE)
        
#         with pd.ExcelWriter(
#             RESULT_FILE,
#             mode='a' if os.path.exists(RESULT_FILE) else 'w',
#             engine='openpyxl',
#             if_sheet_exists='overlay' if os.path.exists(RESULT_FILE) else None
#         ) as writer:
#             df.to_excel(
#                 writer,
#                 index=False,
#                 sheet_name='Results',
#                 header=header,
#                 startrow=writer.sheets['Results'].max_row if not header else 0
#             )
#         batch.clear()
#     except Exception as e:
#         logging.error(f"保存结果失败: {str(e)}")
#         raise

# # 主训练循环
# current_combination = 0
# start_total = time.time()

# try:
#     for lr in HP_CONFIG['lr']:
#         for depth in HP_CONFIG['depth']:
#             for heads in HP_CONFIG['heads']:
#                 for dim_head in HP_CONFIG['dim_head']:
#                     for dropout in HP_CONFIG['dropout']:
#                         for beta in HP_CONFIG['beta']:
#                             for adj_param in HP_CONFIG['adj_parameter']:
#                                 current_combination += 1
#                                 start_iter = time.time()
                                
#                                 # 日志记录当前参数
#                                 param_str = (
#                                     f"lr={lr}, depth={depth}, heads={heads}, "
#                                     f"dim_head={dim_head}, dropout={dropout}, "
#                                     f"beta={beta}, adj_parameter={adj_param} "
#                                     f"({current_combination}/{total_combinations})"
#                                 )
#                                 logging.info(f"开始训练: {param_str}")

#                                 try:
#                                     # 执行训练（假设train_test返回acc, f1_weighted, f1_macro）
#                                     acc, f1_weighted, f1_macro = train_test(  # 修改变量接收
#                                         Methylation_train=Methylation_train,
#                                         miRNASeq_train=miRNASeq_train,
#                                         RNAseq_train=RNAseq_train,
#                                         lr=lr,
#                                         depth=depth,
#                                         heads=heads,
#                                         dim_head=dim_head,
#                                         beta=beta,
#                                         attn_dropout=dropout,
#                                         ff_dropout=dropout,
#                                         classifier_dropout=dropout,
#                                         adj_parameter=adj_param,
#                                         num_epoch_pretrain=1500
#                                     )

#                                     # 缓存结果（修改键名）
#                                     results_cache.append({
#                                         'lr': lr,
#                                         'depth': depth,
#                                         'heads': heads,
#                                         'dim_head': dim_head,
#                                         'dropout': dropout,
#                                         'beta': beta,
#                                         'adj_parameter': adj_param,
#                                         'accuracy': acc,
#                                         'f1_weighted': f1_weighted,  # 修改键名
#                                         'f1_macro': f1_macro          # 新增键
#                                     })

#                                     # 定期保存
#                                     save_results(results_cache)

#                                     # 记录成功信息（修改日志输出）
#                                     elapsed = time.time() - start_iter
#                                     logging.info(
#                                         f"完成: {param_str} | "
#                                         f"耗时: {elapsed:.1f}s | "
#                                         f"结果: ACC={acc:.4f}, F1_weighted={f1_weighted:.4f}, F1_macro={f1_macro:.4f}"  # 修改指标名称
#                                     )

#                                 except torch.cuda.OutOfMemoryError:
#                                     logging.warning(f"显存不足，跳过组合: {param_str}")
#                                     torch.cuda.empty_cache()
#                                 except Exception as e:
#                                     logging.error(f"训练失败: {param_str} | 错误: {str(e)}")
#                                     continue
#                                 # 用于保存指标数据的列表（移除了AUC相关指标）
#                                 train_losses = []
#                                 val_losses = []
#                                 test_losses = []
#                                 val_accuracies = []
#                                 val_f1_weighted_scores = []
#                                 val_f1_macro_scores = []
#                                 test_accuracies = []
#                                 test_f1_weighted_scores = []
#                                 test_f1_macro_scores = []
#                                 train_accuracies = []
#                                 train_f1_weighted_scores = []
#                                 train_f1_macro_scores = []

#     # 最后强制保存剩余结果
#     save_results(results_cache, force_save=True)

# except KeyboardInterrupt:
#     logging.warning("用户中断训练，保存已完成的实验结果...")
#     save_results(results_cache, force_save=True)
# finally:
#     total_time = time.time() - start_total
#     logging.info(f"全部完成! 总耗时: {total_time/3600:.2f} 小时")

# print("实验完成，结果保存在:", RESULT_FILE)

In [ ]:
import time
import torch

# 假设你已经定义好这三个数据集和train_test函数
# from your_module import train_test, Methylation_train, miRNASeq_train, RNAseq_train

# 固定超参数
lr = 0.001
depth = 2
heads = 4
dim_head = 12
dropout = 0.3
beta = 1
adj_param = 8
num_epoch_pretrain = 458

# 开始训练
start_time = time.time()

acc, f1_weighted, f1_macro = train_test(
    Methylation_train=Methylation_train,
    miRNASeq_train=miRNASeq_train,
    RNAseq_train=RNAseq_train,
    lr=lr,
    depth=depth,
    heads=heads,
    dim_head=dim_head,
    beta=beta,
    attn_dropout=dropout,
    ff_dropout=dropout,
    classifier_dropout=dropout,
    adj_parameter=adj_param,
    num_epoch_pretrain=num_epoch_pretrain
)

# 输出结果
elapsed = time.time() - start_time
print(f"训练完成 | 耗时: {elapsed:.2f} 秒")
print(f"准确率: {acc:.4f}")
print(f"F1_weighted: {f1_weighted:.4f}")
print(f"F1_macro: {f1_macro:.4f}")


训练参数：lr=0.0001, depth=6, heads=4, dim_head=32, dropout=0.5, beta=1

Pretrain GCNs...

Training...






































































































































































































































































































































































































































































































































































































































































































































































